In [2]:
library(CBCgrps)
library(nortest)
library(RPostgreSQL)
library(twang)
library(Matching)
library(tidyverse)
library(ggplot2)
library(readxl)
library(tableone)

Loading required package: nortest

Warning message:
"package 'RPostgreSQL' was built under R version 4.3.3"
Loading required package: DBI

Warning message:
"package 'twang' was built under R version 4.3.3"
Warning message in .recacheSubclasses(def@className, def, env):
"undefined subclass "ndiMatrix" of class "replValueSp"; definition not updated"
To reproduce results from prior versions of the twang package, please see the version="legacy" option described in the documentation.

Warning message:
"package 'Matching' was built under R version 4.3.3"
Loading required package: MASS

## 
##  Matching (Version 4.10-14, Build Date: 2023-09-13)
##  See https://www.jsekhon.com for additional documentation.
##  Please cite software as:
##   Jasjeet S. Sekhon. 2011. ``Multivariate and Propensity Score Matching
##   Software with Automated Balance Optimization: The Matching package for R.''
##   Journal of Statistical Software, 42(7): 1-52. 
##


Warning message:
"package 'tidyverse' was built un

In [1]:
data_dir <- file.path("..", "data")

In [3]:
data <- read_excel(file.path(data_dir, "baseline.xlsx"))

## 以下是二组的baseline分析比较

In [4]:
vec1 <- as.data.frame(data)

In [5]:
# Rename the 'CK-MB' column to 'CK_MB'
names(vec1)[names(vec1) == 'CK-MB'] <- 'CK_MB'

features <- c("los_hos", "serum", "TCM", "gender", "age", "weight", "hight", "IBM", 
              "ALT", "AST", "AKI", "complication", "GCs", "dose", "PT", "APTT",
              "fib", "DD", "PLT", "CK", "CKMB", "Tn")

In [6]:

tab1 <-multigrps(vec1, gvar = "TCM")
print(tab1, quote = T)

                                                                        
1                "Variables"       "Total (n = 300)"        "0 (n = 73)"
2  "stay_id, Median (Q1,Q3)" "150.5 (75.75, 225.25)"     "135 (74, 203)"
3  "los_hos, Median (Q1,Q3)"         "12 (7, 22.25)"        "12 (9, 23)"
4             "serum, n (%)"                      ""                  ""
5                      "  0"                "10 (3)"             "2 (3)"
6                      "  1"              "181 (60)"           "44 (60)"
7                      "  2"               "49 (16)"           "17 (23)"
8                      "  3"               "60 (20)"           "10 (14)"
9            "gender, n (%)"                      ""                  ""
10                     "  1"              "169 (56)"           "39 (53)"
11                     "  2"              "131 (44)"           "34 (47)"
12     "age, Median (Q1,Q3)"        "58 (46.75, 66)"       "59 (53, 66)"
13  "weight, Median (Q1,Q3)"        "63 (53.75, 69)

In [8]:
tab <- CreateTableOne(vars = features,
                      strata = "TCM",
                      data = vec1,
                      argsNormal = list(var.equal = FALSE))


In [9]:
capture.output(tab_df <- tab %>%
               print(smd = TRUE) %>%
               as.data.frame(stringsAsFactors = FALSE) %>%
               tibble::rownames_to_column(var = "n") %>%
               dplyr::select(n, everything())) %>% invisible

tab_df



n,0,1,p,test,SMD
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
n,73,227,,,
los_hos (mean (SD)),14.47 (8.12),14.06 (9.66),0.721,,0.046
serum (mean (SD)),1.48 (0.77),1.55 (0.87),0.532,,0.081
TCM (mean (SD)),0.00 (0.00),1.00 (0.00),NaN,,Inf
gender (mean (SD)),1.47 (0.50),1.43 (0.50),0.569,,0.077
age (mean (SD)),55.66 (15.98),52.78 (18.30),0.198,,0.168
weight (mean (SD)),61.34 (11.11),61.24 (12.74),0.950,,0.008
hight (mean (SD)),1.64 (0.11),1.64 (0.11),0.882,,0.020
IBM (mean (SD)),22.78 (3.00),22.61 (3.49),0.673,,0.055


In [10]:
data.table::fwrite(tab_df, file.path(data_dir, "tab_baseline.csv"), col.names = TRUE)

In [11]:
tab_serum <- CreateTableOne(vars = features,
                      strata = "serum",
                      data = vec1,
                      argsNormal = list(var.equal = FALSE))


In [12]:
capture.output(tab_df_serum <- tab_serum %>%
               print(smd = TRUE) %>%
               as.data.frame(stringsAsFactors = FALSE) %>%
               tibble::rownames_to_column(var = "n") %>%
               dplyr::select(n, everything())) %>% invisible

tab_df_serum



n,0,1,2,3,p,test,SMD
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
n,10,181,49,60,,,
los_hos (mean (SD)),13.80 (13.63),14.49 (9.37),13.35 (7.77),13.86 (9.58),0.851,,0.061
serum (mean (SD)),0.00 (0.00),1.00 (0.00),2.00 (0.00),3.00 (0.00),NaN,,Inf
TCM (mean (SD)),0.80 (0.42),0.76 (0.43),0.65 (0.48),0.83 (0.38),0.220,,0.224
gender (mean (SD)),1.50 (0.53),1.47 (0.50),1.29 (0.46),1.45 (0.50),0.126,,0.226
age (mean (SD)),56.50 (13.61),49.67 (19.05),59.33 (14.92),59.67 (13.14),<0.001,,0.341
weight (mean (SD)),69.30 (10.21),60.97 (13.19),61.00 (10.52),61.03 (11.18),0.124,,0.381
hight (mean (SD)),1.65 (0.05),1.63 (0.12),1.65 (0.07),1.64 (0.09),0.651,,0.096
IBM (mean (SD)),25.48 (2.91),22.62 (3.44),22.38 (3.24),22.49 (3.21),0.033,,0.505


In [13]:
data.table::fwrite(tab_df_serum, file.path(data_dir, "tab_baseline_serum.csv"), col.names = TRUE)

## 以下是二组的住院中期的分析比较

In [17]:
subgroup2 <- as.data.frame(readxl::read_excel(file.path(data_dir, "subgroup2.xlsx")))

In [18]:
colnames(subgroup2)

[1] "serum"          "TCM"            "hight"          "IBM"           
 [5] "ALT"            "AST"            "AST1"           "AKI"           
 [9] "complication"   "GCs"            "Accumulatedose" "dose"          
[13] "doseday"        "los_hos"        "PT"             "APTT"          
[17] "fib"            "DD"             "PLT"            "CK"            
[21] "CKMB"           "Tn"

In [19]:
features <- c("los_hos", "serum", "TCM", "gender", "age", "weight", "hight", "IBM", 
              "ALT", "AST", "AKI", "complication", "GCs", "dose","PT", "APTT",
              "fib", "DD", "PLT", "CK", "CKMB", "Tn")

In [20]:
tab_subgroup2 <- CreateTableOne(vars = features,
                      strata = "TCM",
                      data = subgroup2,
                      argsNormal = list(var.equal = FALSE))


Warning message in ModuleReturnVarsExist(vars, data):
"The data frame does not have: gender age weight  Dropped"


In [21]:
capture.output(tab_df_subgroup2 <- tab_subgroup2 %>%
               print(smd = TRUE) %>%
               as.data.frame(stringsAsFactors = FALSE) %>%
               tibble::rownames_to_column(var = "n") %>%
               dplyr::select(n, everything())) %>% invisible

tab_df_subgroup2


n,0,1,p,test,SMD
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
n,73,227,,,
los_hos (mean (SD)),8.16 (2.03),6.00 (1.40),<0.001,,1.242
serum (mean (SD)),1.48 (0.77),1.55 (0.87),0.532,,0.081
TCM (mean (SD)),0.00 (0.00),1.00 (0.00),NaN,,Inf
hight (mean (SD)),1.64 (0.11),1.64 (0.11),0.882,,0.020
IBM (mean (SD)),22.78 (3.00),22.61 (3.49),0.673,,0.055
ALT (mean (SD)),66.77 (66.51),46.97 (42.10),0.019,,0.356
AST (mean (SD)),31.23 (20.58),27.41 (17.81),0.157,,0.198
AKI (mean (SD)),0.12 (0.33),0.09 (0.28),0.422,,0.113


In [22]:
data.table::fwrite(tab_df_subgroup2, file.path(data_dir, "tab_subgroup2.csv"), col.names = TRUE)

## 以下是二组第三阶段的分析

In [24]:
subgroup3 <- as.data.frame(readxl::read_excel(file.path(data_dir, "subgroup3.xlsx")))


In [25]:
features <- c("los_hos", "serum", "group", "gender", "age", "weight", "hight", "IBM", 
              "ALT", "AST", "AKI", "complication", "GCs", "dose","Accumulatedose", "PT", "APTT",
              "fib", "DD", "PLT", "CK", "CKMB", "Tn")

In [26]:
tab_subgroup3 <- CreateTableOne(vars = features,
                      strata = "TCM",
                      data = subgroup3,
                      argsNormal = list(var.equal = FALSE))

Warning message in ModuleReturnVarsExist(vars, data):
"The data frame does not have: group gender age weight  Dropped"


In [27]:
capture.output(tab_df_subgroup3 <- tab_subgroup3 %>%
               print(smd = TRUE) %>%
               as.data.frame(stringsAsFactors = FALSE) %>%
               tibble::rownames_to_column(var = "n") %>%
               dplyr::select(n, everything())) %>% invisible

tab_df_subgroup3

n,0,1,p,test,SMD
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
n,73,227,,,
los_hos (mean (SD)),8.16 (2.03),6.00 (1.40),<0.001,,1.242
serum (mean (SD)),1.48 (0.77),1.55 (0.87),0.532,,0.081
hight (mean (SD)),1.64 (0.11),1.64 (0.11),0.882,,0.020
IBM (mean (SD)),22.78 (3.00),22.61 (3.49),0.673,,0.055
ALT (mean (SD)),66.77 (66.51),46.97 (42.10),0.019,,0.356
AST (mean (SD)),31.23 (20.58),27.41 (17.81),0.157,,0.198
AKI (mean (SD)),0.12 (0.33),0.09 (0.28),0.422,,0.113
complication (mean (SD)),0.51 (0.82),0.44 (0.86),0.553,,0.079


## 以下是对第三次阶段的数据进行差值分析

In [28]:
subgroup2 <- as.data.frame(readxl::read_excel(file.path(data_dir, "subgroup2.xlsx")))

In [29]:
# add workspace
subgroup2 <- multigrps(subgroup2, gvar = "TCM", workspace = 5e8) 
print(subgroup2, quote = T)


                                                                              
1                       "Variables"     "Total (n = 300)"         "0 (n = 73)"
2                    "serum, n (%)"                    ""                   ""
3                             "  0"              "10 (3)"              "2 (3)"
4                             "  1"            "181 (60)"            "44 (60)"
5                             "  2"             "49 (16)"            "17 (23)"
6                             "  3"             "60 (20)"            "10 (14)"
7           "hight, Median (Q1,Q3)"     "1.65 (1.6, 1.7)"    "1.65 (1.6, 1.7)"
8                  "IBM, Mean ± SD"        "22.65 ± 3.38"          "22.78 ± 3"
9             "ALT, Median (Q1,Q3)"      "36 (23, 63.25)"        "42 (25, 77)"
10            "AST, Median (Q1,Q3)"         "23 (16, 35)"        "25 (17, 37)"
11           "AST1, Median (Q1,Q3)"         "26 (21, 39)"        "25 (20, 33)"
12                     "AKI, n (%)"                 

In [34]:
write.csv(subgroup2, file = file.path(data_dir, "subgroup2.csv"), fileEncoding = "GBK", row.names = FALSE)


In [35]:
subgroup3 <- as.data.frame(readxl::read_excel(file.path(data_dir, "subgroup3.xlsx")))

In [36]:
# add workspace
subgroup3 <- multigrps(subgroup3, gvar = "TCM", workspace = 5e8) 
print(subgroup3, quote = T)


                                                           
1                       "Variables"       "Total (n = 300)"
2         "stay_id, Median (Q1,Q3)" "150.5 (75.75, 225.25)"
3                    "serum, n (%)"                      ""
4                             "  0"                "10 (3)"
5                             "  1"              "181 (60)"
6                             "  2"               "49 (16)"
7                             "  3"               "60 (20)"
8           "hight, Median (Q1,Q3)"       "1.65 (1.6, 1.7)"
9                  "IBM, Mean ± SD"          "22.65 ± 3.38"
10            "ALT, Median (Q1,Q3)"        "36 (23, 63.25)"
11            "AST, Median (Q1,Q3)"           "23 (16, 35)"
12           "AST1, Median (Q1,Q3)"           "26 (21, 39)"
13                     "AKI, n (%)"                      ""
14                            "  0"              "270 (90)"
15                            "  1"               "29 (10)"
16            "complication, n (%)"     

In [37]:
write.csv(subgroup3, file = file.path(data_dir, "subgroup3.csv"), fileEncoding = "GBK", row.names = FALSE)

In [38]:
library(tableone)
library(tidyverse)

In [39]:
fml <- "TCM ~ los_hos + serum + IBM + 
      AKI + GCs + PT + APTT +
       fib + DD + PLT + CK + CKMB + Tn"
subgroup2 <- as.data.frame(readxl::read_excel(file.path(data_dir, "subgroup2.xlsx")))

In [40]:
unweighted <- glm(as.formula(fml), data = subgroup2, family = binomial, na.action = na.exclude)


In [41]:
summary(unweighted)


Call:
glm(formula = as.formula(fml), family = binomial, data = subgroup2, 
    na.action = na.exclude)

Coefficients:
              Estimate Std. Error z value Pr(>|z|)    
(Intercept)  7.4057832  1.6488620   4.491 7.07e-06 ***
los_hos     -0.9236465  0.1295840  -7.128 1.02e-12 ***
serum        0.0405586  0.2061297   0.197    0.844    
IBM         -0.0116229  0.0530914  -0.219    0.827    
AKI          0.4734466  0.6037267   0.784    0.433    
GCs          0.1176638  0.2978486   0.395    0.693    
PT          -0.0124996  0.0095470  -1.309    0.190    
APTT         0.0064976  0.0085488   0.760    0.447    
fib         -0.0260836  0.1058377  -0.246    0.805    
DD           0.0069835  0.0121384   0.575    0.565    
PLT         -0.0045171  0.0035396  -1.276    0.202    
CK          -0.0001308  0.0001776  -0.737    0.461    
CKMB        -0.0009629  0.0016381  -0.588    0.557    
Tn          -7.9952123  3.2173586  -2.485    0.013 *  
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.'